In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Convert data to numpy arrays
x_data = data[[ 'open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D



# Convert data to Tensors
x_feature_tensors = torch.tensor(x_data, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data, dtype=torch.float32)


# Split the data into training and testing sets
train_size = int(len(x_feature_tensors) * 0.67)
x_train, x_test = x_feature_tensors[:train_size], x_feature_tensors[train_size:]
y_train, y_test = y_feature_tensors[:train_size], y_feature_tensors[train_size:]



# Initialize the scaler
scaler = MinMaxScaler()

# fitting the MinMaxScaler to training data using fit_transform and then
#  test data only transform
x_train_scaled = scaler.fit_transform(x_train.detach().numpy())
x_test_scaled = scaler.transform(x_test.detach().numpy())  # Use transform, not fit_transform

y_train_scaled = scaler.fit_transform(y_train.detach().numpy())
y_test_scaled = scaler.transform(y_test.detach().numpy())  # Use transform, not fit_transform





In [3]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train_scaled) - window_size
batch_size = 1
input_size = x_train_scaled.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
import random
import pandas as pd  # Import pandas for handling data rows

# Assuming x_train_scaled, y_train_scaled, x_test_scaled, y_test_scaled, input_size, output_size, test_window_size, scaler are available

# Splitting the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train_scaled, y_train_scaled, test_size=0.2, shuffle=False)

# Hyperparameter search space
search_space = {
    'learning_rate': [0.00001, 0.00005, 0.0001],
    'window_size': [3, 5, 6],
    'hidden_dim': [64, 128, 256],
    'n_layers': [10, 11, 12],
    'batch_evaluation_frequency': [4, 5, 6, ]
}

# Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 128,
# 'n_layers': 11, 'batch_evaluation_frequency': 7}

# Best Parameters: {'learning_rate': 0.0001, 'window_size': 10, 'hidden_dim': 256,
# 'n_layers': 10, 'batch_evaluation_frequency': 5}

best_predicted_error = float('inf')
best_params = None

# Number of random search iterations
num_iterations = 10

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

            # Predict and calculate error on a specific data row
            with torch.no_grad():
                last_test_window = x_test_scaled[-test_window_size:]
                last_test_window_tensor = torch.tensor(last_test_window, dtype=torch.float32).view(1, test_window_size, input_size)
                hidden = model.init_hidden(1)
                predicted_close_scaled, _ = model(last_test_window_tensor, hidden)

                predicted_close = scaler.inverse_transform(predicted_close_scaled.detach().numpy())
                real_value = pd.read_csv("../data/data/aapl_raw_data.csv").loc[10748, 'close']
                predicted_error = abs(real_value - predicted_close[0][0])

                if predicted_error < best_predicted_error:
                    best_predicted_error = predicted_error
                    best_params = params

print(f"Best Parameters: {best_params}")
print(f"Smallest Predicted Error: {best_predicted_error}")

# Print the best predicted close value
with torch.no_grad():
    last_test_window = x_test_scaled[-test_window_size:]
    last_test_window_tensor = torch.tensor(last_test_window, dtype=torch.float32).view(1, test_window_size, input_size)
    hidden = model.init_hidden(1)
    predicted_close_scaled, _ = model(last_test_window_tensor, hidden)

    predicted_close = scaler.inverse_transform(predicted_close_scaled.detach().numpy())
    best_predicted_close = predicted_close[0][0]
    print(f"Best Predicted Close Value: {best_predicted_close}")
